In [26]:
import numpy as np
from scipy import spatial 
import faiss
from time import time
import matplotlib.pyplot as plt
from collections import defaultdict

## Helper Function

In [27]:
def semi_optimized_exhaustive_search(
        index_vectors: np.ndarray,
        query_vectors: np.ndarray,
        k: int,
):
    """
    This function performs an optimized exhaustive search.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        query_vectors: An array of shape (n_queries, dim) containing the query vectors. 
        dim: The dimensionality of the vectors.
    Returns:
        An array of shape (n_queries, k) containing the indices of the k nearest neighbors for each query vector.
    """
    ann_lists = []
    for query_vec in query_vectors:
        distances = np.linalg.norm(index_vectors - query_vec, axis=1)
        ann_lists.append(list(np.argsort(distances)[:k]))
    return np.array(ann_lists)

In [28]:
def build_faiss_flatl2_index(
        index_vectors: np.ndarray,
        dim: int,
):
    """
    This function builds a Faiss flat L2 index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors. 
    Returns:
        A Faiss flat L2 index.
    """
    index = faiss.IndexFlatL2(dim)
    index.add(index_vectors)
    return index

In [29]:
def faiss_search(
        query_vectors: np.ndarray,
        index: faiss.Index,
        k: int,
):
    """
    This function uses a Faiss index to search for the k-nearest neighbors of query_vectors.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors. 
        index: A Faiss index.
        k: The number of nearest neighbors to retrieve.
    Returns:
        An array of shape (, ) containing the indices of the k-nearest neighbors for each query vector.
    """
    distances, indices = index.search(query_vectors, k)
    return indices

In [30]:
def build_faiss_lsh_index(
        index_vectors: np.ndarray,
        dim: int,
        nbits: int,
):
    """
    This function builds a Faiss LSH index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors. 
        nbits: The number of bits to use in the hash.
    Returns:
        A Faiss LSH index.
    """
    index = faiss.IndexLSH(dim, nbits)
    index.add(index_vectors)
    return index

In [31]:
def compute_recall_at_k(
        nn_gt: np.ndarray,
        ann: np.ndarray,
        k: int,
):
    """
    This function computes the recall@k.
    Args:
        nn_gt: The ground truth nearest neighbors.
        ann: The approximate nearest neighbors.
        k: The number of nearest neighbors to consider.
    Returns:
        The recall@k.
    """
    return round(sum([len(set(ann[i]) & set(nn_gt[i])) / k for i in range(len(ann))])/len(ann), 3)

# 2.1 -- LSH vs Naive Exhaustive Search (Regular Index Vectors)
### You just have to run the following cells and add the following results to the report:
* running time of the ground truth computation with semi_optimized_exhaustive_search (wall time)
* running time of creating faiss_lsh_index (wall time)
* running time of faiss_search over query_vectors with faiss_lsh_index (wall time)
* recall@10 for faiss_lsh_ann

In [32]:
query_vectors = np.load('data/query_vectors.npy')
index_vectors = np.load('data/index_vectors.npy')
k=10
dim = index_vectors.shape[1]

In [33]:
%%time
gt_nn = semi_optimized_exhaustive_search(index_vectors, query_vectors, k)

CPU times: total: 22.1 s
Wall time: 26.8 s


In [34]:
%%time
faiss_lsh_index = build_faiss_lsh_index(index_vectors, dim, nbits=2000)

CPU times: total: 18 s
Wall time: 5.54 s


In [35]:
%%time
faiss_lsh_ann = faiss_search(query_vectors, faiss_lsh_index, k)

CPU times: total: 1.02 s
Wall time: 187 ms


In [36]:
print(f"recall@10 for faiss_lsh_index: {compute_recall_at_k(gt_nn, faiss_lsh_ann, k)}")

recall@10 for faiss_lsh_index: 0.138


# 2.2 -- Custom Indexing Algorithm
Build an indexing algorithm that satisfies the following requirements:
* The indexing algorithm should be able to handle vectors of different dimensions
* The running time of the indexing should be less than half of the running time of semi_optimized_exhaustive_search), reported in Section 2.1.
* The running time of searching over the index should be less than a third (1/3) of the time of the semi_optimized_exhaustive_search function, reported in Section 2.1.
* The performance (in terms of recall@10) of the indexing algorithm should be at least 0.8.

The last three bullets should also appear in the report.
You are allowed to add as many helper functions as you need. You cannot use faiss of scipy libraries for this task. Numpy is allowed. 

You can also test your algorithm with the additional two query-index sets by replacing the calls made few cells ago to:

    query_vectors = np.load('data/query_vectors2.npy')
    index_vectors = np.load('data/index_vectors2.npy')
or:

    query_vectors = np.load('data/query_vectors3.npy')
    index_vectors = np.load('data/index_vectors3.npy')
    
the aforementioned requirements should also be satisfied over these two query-index sets. No need to insert the results over these two to the report.

In [37]:
# Write your code for 2.2.2 here
# You are allowed to add more arguments to the functions and create more functions if needed.

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

class PCA_Kmeans:
    def __init__(self, min_clusters=7, max_clusters=15):
        # Initialize the PCA_Kmeans class with the min_clusters and max_clusters
        """
        This function initializes the PCA_Kmeans class. 
        This class will perform PCA to reduce the dimensionality of the index vectors, and then use KMeans to cluster the reduced vectors.
        Args:
            min_clusters: The minimum number of clusters to consider.
            max_clusters: The maximum number of clusters to consider.            
        """
        self.min_clusters = min_clusters
        self.max_clusters = max_clusters
        self.kmeans = None
        self.reduced_vectors = None
        self.original_vectors = None
        self.pca = None
        self.cluster_map = defaultdict(list)
        self.cluster_centers = None

    def fit(self, index_vectors):
        """
        This function fits the PCA and KMeans models to the index vectors.
        Args:
            index_vectors: An array of shape (n_index, dim) containing the index vectors.
        """
        self.original_vectors = np.array(index_vectors)
        self._perform_pca()
        self._find_optimal_clusters_and_kmeans()

    def _perform_pca(self):
        """
        This function performs PCA on the original vectors.
        """
        self.pca = PCA(n_components=2)
        self.reduced_vectors = self.pca.fit_transform(self.original_vectors)

    def _find_optimal_clusters_and_kmeans(self):
        """
        This function finds the optimal number of clusters and fits the KMeans model.
        """
        sse, kmeans_models = self._calculate_sse_and_models()
        elbow = self._find_elbow(sse)
        self.kmeans = kmeans_models[elbow - self.min_clusters]
        self._assign_clusters()

    def _calculate_sse_and_models(self):
        """
        This function calculates the sum of squared errors for different numbers of clusters.
        Returns:
            sse: A list containing the sum of squared errors for different numbers of clusters.
            kmeans_models: A list containing the KMeans models for different numbers of clusters.
        """
        sse, kmeans_models = [], []
        for k in range(self.min_clusters, self.max_clusters + 1):
            kmeans = KMeans(n_clusters=k).fit(self.reduced_vectors)
            sse.append(kmeans.inertia_)
            kmeans_models.append(kmeans)
        return sse, kmeans_models

    def _find_elbow(self, sse):
        """
        This function finds the elbow point in the SSE curve.
        Args:
            sse: A list containing the sum of squared errors for different numbers of clusters.
        Returns:
            The optimal number of clusters.
        """
        diff2 = np.diff(np.diff(sse))
        return np.argmax(diff2) + self.min_clusters + 1

    def _assign_clusters(self):
        """
        This function assigns the vectors to clusters.
        """
        self.cluster_centers = self.kmeans.cluster_centers_
        cluster_labels = self.kmeans.labels_
        for idx, label in enumerate(cluster_labels):
            self.cluster_map[label].append(idx)
    
    def query(self, query_vector, k):
        """
        This function finds the k-nearest neighbors for a query vector.
        Args:
            query_vector: The query vector.
            k: The number of nearest neighbors to retrieve.
        Returns:
            A list containing the indices of the k-nearest neighbors.
        """
        reduced_query = self.pca.transform([query_vector])[0]
        nearest_cluster_idx = self._find_nearest_cluster_idx(reduced_query)
        candidate_indices = self.cluster_map[nearest_cluster_idx]
        return self._get_k_nearest_neighbors(query_vector, candidate_indices, k)

    def _find_nearest_cluster_idx(self, reduced_query):
        """
        This function finds the nearest cluster to a query vector.
        Args:
            reduced_query: The reduced query vector.
        Returns:
            The index of the nearest cluster.
        """
        distances = np.linalg.norm(self.cluster_centers - reduced_query, axis=1)
        return np.argmin(distances)

    def _get_k_nearest_neighbors(self, query_vector, candidate_indices, k):
        """
        This function finds the k-nearest neighbors from a list of candidate indices.
        Args:
            query_vector: The query vector.
            candidate_indices: A list of candidate indices.
            k: The number of nearest neighbors to retrieve.
        Returns:
            A list containing the indices of the k-nearest neighbors.
        """
        distances = np.linalg.norm(self.original_vectors[candidate_indices] - query_vector, axis=1)
        nearest_indices = np.argsort(distances)[:k]
        return [candidate_indices[i] for i in nearest_indices]

    def queries(self, query_vectors, k):
        """
        This function finds the k-nearest neighbors for multiple query vectors.
        Args:
            query_vectors: An array of shape (n_queries, dim) containing the query vectors.
            k: The number of nearest neighbors to retrieve.
        Returns:
            A list of lists containing the indices of the k-nearest neighbors for each query vector.
        """
        return [self.query(query_vector, k) for query_vector in query_vectors]
            

def custom_indexing_algorithm(index_vectors, dim):
    """
    This function builds an index from scratch.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors.
    Returns:
        An index.
    """
    idx = PCA_Kmeans()
    idx.fit(index_vectors)
    return idx


def custom_index_search(query_vectors, index, k):
    """
    This function searches over the custom index.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors.
        index: The custom index.
        k: The number of nearest neighbors to retrieve.
    """
    gt_nn_pca = index.queries(query_vectors, k)
    return gt_nn_pca
    

In [38]:
# Add hyperparameters here (if needed)

In [39]:
%%time
custom_index = custom_indexing_algorithm(index_vectors, dim)

CPU times: total: 547 ms
Wall time: 469 ms


In [40]:
%%time
custom_index_ann = custom_index_search(query_vectors, custom_index, k)

CPU times: total: 1.77 s
Wall time: 2.09 s


In [41]:
print(f"recall@10 for custom_index_search: {compute_recall_at_k(gt_nn, custom_index_ann, k)}")

recall@10 for custom_index_search: 1.0
